In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm, datasets
from sklearn.calibration import CalibratedClassifierCV

# загружаем данные

columns=["message","class"]
data = pd.read_csv("../resources/extracted/data.csv", sep=",", header=None, names=columns)

# разбиваем данные на тренировочный и тестовый наборы
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

# преобразуем текст сообщений в векторы признаков
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data['message'])
X_test = vectorizer.transform(test_data['message'])

# задаем целевую переменную
y_train = train_data['class']
y_test = test_data['class']

# обучаем модель SVM
clf = svm.SVC(kernel='linear', probability=True, random_state=42)
clf.fit(X_train, y_train)

# Получаем вероятности классов
calibrated_svm = CalibratedClassifierCV(clf, cv='prefit')
calibrated_svm.fit(X_train, y_train)
calibrated_svm.predict_proba(X_test)

# оцениваем точность с вероятностью
accuracy = calibrated_svm.score(X_test, y_test)
print("Calibrated accuracy:", accuracy)

# оцениваем точность модели
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)


Exception: Requested setting SECRETS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.
Calibrated accuracy: 0.9132653061224489
Accuracy: 0.9132653061224489


In [24]:
#Проверка модели
mine_data = {'message': ['Можешь мне поставить напоминание на 17:30',],}
#mine_data = test_data.reset_index(drop=True)

mine_df = pd.DataFrame(mine_data)
mine_test = vectorizer.transform(mine_df['message'])
probabilities = np.array(calibrated_svm.predict_proba(mine_test))
labels = np.array(calibrated_svm.predict(mine_test))
classes = np.array(calibrated_svm.classes_)
#print(classes)
#print(labels)
#for i in range(len(labels)):
#    print(mine_df['message'][i])
#    print(labels[i])
#    print()

def predict_class():
    mine_df = pd.DataFrame({'message': ['Можешь мне поставить напоминание на 17:30']})
    mine_test = vectorizer.transform(mine_df['message'])
    probabilities = np.array(calibrated_svm.predict(mine_test))
    return probabilities[0]

predict_class()

'reminder'

In [17]:
# Оценка вероятностей тестов
print(mine_data['message'])
counter = 0
for c, p in zip(labels, probabilities):
    print(f"{mine_df['message'][counter]}")
    counter += 1
    print(f"{c}:")
    for i, pr in enumerate(p):
        print(f"\t- {classes[i]}: {pr*100:.2f}%")
    print()


0          Добавить встречу с клиентом на завтра в 10:00
1                                        Встреча в 15:00
2                 Напомнить мне о важной встрече в 12:00
3                          Сохранить фотографии в облако
4          Напомнить мне о медицинской процедуре в 17:00
                             ...                        
191    Я накоплю достаточно денег чтобы взять отпуск ...
192    Я хочу научиться делать сплит в течение следую...
193    Я начну ежедневную практику медитации и буду п...
194    Я хочу открыть свою собственную линию одежды в...
195    Я хочу пройти 30-дневный фитнес-зачет и в конц...
Name: message, Length: 196, dtype: object
Добавить встречу с клиентом на завтра в 10:00
calender:
	- appointment: 21.15%
	- calender: 78.85%
	- chat: 0.00%
	- event: 0.00%
	- goal: 0.00%
	- intention: 0.00%
	- reminder: 0.00%
	- save: 0.00%

Встреча в 15:00
reminder:
	- appointment: 17.59%
	- calender: 1.12%
	- chat: 0.00%
	- event: 0.00%
	- goal: 0.00%
	- intention: 0.0

In [20]:
# Save Model Using Pickle
import pickle

# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))

# save the vectorizer to disk
filename_vectorizer = 'vectorizer.sav'
pickle.dump(vectorizer, open(filename_vectorizer, 'wb'))

# some time later...

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
loaded_vectorizer = pickle.load(open(filename_vectorizer, 'rb'))

mine_data = {'message': ['Можешь мне поставить напоминание на 17:30',],}

mine_df = pd.DataFrame(mine_data)
mine_test = loaded_vectorizer.transform(mine_df['message'])
probabilities = np.array(loaded_model.predict(mine_test))
#result = loaded_model.score(X_test, y_test)
print(probabilities)

['reminder']
